In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping

from sklearn.metrics import fbeta_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import cv2
from tqdm import tqdm

from datetime import datetime
import time
import configparser
import json
import sys

from utils.file import makedirs
from utils.recorder import record_model_medata, record_model_scores
from utils.loader import *
from utils.f2thresholdfinder import *
from utils.imagegen import *
from utils.models import *
from utils.custommetrics import *
from utils.samplesduplicator import duplicate_train_samples
from utils.training import *
from utils.predictor import submission_dataframe


Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
c:\users\me\appdata\local\temp\try_flags_scquds.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
timestr = time.strftime("%Y%m%d-%H%M%S")
start_time = datetime.now()

In [3]:
config_file = 'cfg/default.cfg'

# command line args processing "python aggregate_model.py cfg/3.cfg"
if len(sys.argv) > 1 and '.cfg' in sys.argv[1]:
    config_file = sys.argv[1]

print('reading configurations from config file: {}'.format(config_file))

settings = configparser.ConfigParser()
settings.read(config_file)
data_dir = settings.get('data', 'data_dir')

df_train = pd.read_csv(data_dir + 'train_v2.csv')
model_filename = 'aggregate_model_'+ timestr +'.h5'
model_filepath = data_dir + 'models/' + model_filename
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
number_of_samples = len(df_train.index)
print('total number of samples: {}'.format(number_of_samples))

# WARNING: keras allow either 1, 3, or 4 channels per pixel. Other numbers not allowed.
data_mask_label = np.array(['R', 'G', 'B', 'NDVI', 'NDWI', 'NIR'])
#print(settings.get('data', 'data_mask'))
data_mask_list = json.loads(settings.get('data', 'data_mask'))

data_mask = ma.make_mask(data_mask_list)
print(data_mask)

num_channels = np.sum(data_mask)
need_norm_stats = False

model_id = settings.get('model', 'model_id')
print('model: {}'.format(model_id))

# default to 64
rescaled_dim = 64
if settings.has_option('data', 'rescaled_dim'):
    rescaled_dim = settings.getint('data', 'rescaled_dim')
print('rescaled dimension: {}'.format(rescaled_dim))

# one epoch is an arbitrary cutoff : one pass over the entire training set
number_epoch = settings.getint('model', 'number_epoch')

# a batch results in exactly one update to the model.
# batch_size is limited by model size and GPU memory
batch_size = settings.getint('model', 'batch_size') 
print('batch size: {}'.format(batch_size))

classifier_threshold = 0.2 # used for end of epoch f2 approximation only

split = int(number_of_samples * 0.80)  # TODO we may want to increase to 0.90 eventually
number_validations = number_of_samples - split

has_augmentation_config = settings.has_section('augmentation')
if has_augmentation_config:
    rotation_range = settings.getint('augmentation', 'rotation_range')
    horizontal_flip = settings.getboolean('augmentation', 'horizontal_flip')
    vertical_flip = settings.getboolean('augmentation', 'vertical_flip')
    print('rotation_range:{} horizontal_flip:{} vertical_flip:{}'.format(rotation_range,horizontal_flip,vertical_flip))


reading configurations from config file: cfg/default.cfg


C:\Users\Me\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


total number of samples: 40479
[ True  True  True  True False False]
model: JAGG_2
rescaled dimension: 64
batch size: 512
rotation_range:20 horizontal_flip:True vertical_flip:True


In [4]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

print(labels)
print(len(labels))

['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']
17


In [5]:
#label_map = {l: i for i, l in enumerate(labels)}
#inv_label_map = {i: l for l, i in label_map.items()}

In [30]:
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

(40479L, 64L, 64L, 6L)
(40479L, 17L)


In [31]:
x_train = x_train[:, :, :, data_mask]

x_train = x_train.transpose(0,3,1,2)  # https://github.com/fchollet/keras/issues/2681
print(x_train.shape)

(40479L, 4L, 64L, 64L)


In [32]:
# TODO save the shuffling order to hdf5 so we can recreate the training and validation sets post execution.

# shuffle the samples because 
# 1) the original samples may not be randomized & 
# 2) to avoid the possiblility of overfitting the validation data while we tune the model
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

In [33]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(32383L, 4L, 64L, 64L)
(32383L, 17L)
(8096L, 4L, 64L, 64L)
(8096L, 17L)


In [34]:
# experimental hack to get more samples for augmentations for a specific low-frequency tag in unbalanced dataset. e.g. habitation
# selecting the optimal multiplier is sensitive

augmentation_hack_config = settings.has_section('augmentation_hack')
if augmentation_hack_config:
    dup_multiplier = settings.getint('augmentation_hack', 'multiplier')
    hack_label_target = settings.get('augmentation_hack', 'label_target')

if augmentation_hack_config:
    x_train, y_train = duplicate_train_samples(x_train, y_train, labels.index(hack_label_target), multiplier=dup_multiplier)
    print(x_train.shape)
    print(y_train.shape)


(44179L, 4L, 64L, 64L)
(44179L, 17L)


In [35]:
# dynamicly set num_samples_per_epoch
# TODO understand the implications of num_samples_per_epoch.
# +0.002 to +0.01??? F2 score improvement when number_of_samples * 3
# num_samples_per_epoch = 1000
num_samples_per_epoch = x_train.shape[0]

In [36]:
single_taget_model = False
# warning: experimental. 
# shuffling won't let you put things back together
if settings.has_option('data', 'single_target'):
    single_taget_model = True
    single_target_label = settings.get('data', 'single_target')
    single_target_label_index = labels.index(single_target_label)
    y_train = y_train[:,single_target_label_index]
    y_valid = y_valid[:,single_target_label_index]
    
score_averaging_method = 'binary' if single_taget_model else 'samples'

In [37]:
def get_img_generator():
    if has_augmentation_config:
        return GeneralImgGen(rotation_range = rotation_range, 
                             horizontal_flip = horizontal_flip, 
                             vertical_flip = vertical_flip)
    else:
        return ScaledDown() # default
    
image_generator = get_img_generator()
print('image generator', image_generator)

('image generator', rotation_range:20 horizontal_flip:True vertical_flip:True)


In [38]:
# this is the augmentation configuration we will use for training
train_datagen = image_generator.getTrainGenenerator()

In [39]:
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    train_datagen.fit(x_train)

In [40]:
train_generator = train_datagen.flow(
        x_train,
        y_train,
        batch_size=batch_size,
        shuffle=True) 

In [41]:
validation_datagen = image_generator.getValidationGenenerator()

In [42]:
# workaround to provide your own stats: 
# http://stackoverflow.com/questions/41855512/how-does-data-normalization-work-in-keras-during-prediction/43069409#43069409
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    validation_datagen.fit(x_valid)

In [43]:
validation_generator = validation_datagen.flow(
        x_valid,
        y_valid,
        batch_size=batch_size,
        shuffle=False)

In [44]:
if single_taget_model:
    set_model_output_layer_size(1)
    
model = get_model(model_id, num_channels, rescaled_dim, rescaled_dim)

# https://github.com/fchollet/keras/issues/369
# https://github.com/fchollet/keras/blob/master/keras/losses.py
model.compile(loss='binary_crossentropy', # Is this the best loss function?
              optimizer='adam',
              metrics=['accuracy', 'recall', 'precision'])

In [45]:
# BUG when resuming training, the learning rate need to be decreased.
# let's load an existing trained model and continue training more epoch gives 0.01 improvement in LB score.
# model = load_model(data_dir + 'models/aggregate_model_20170507-124128.h5') # 0.86
# model = load_model(data_dir + 'models/aggregate_model_20170507-184232.h5') # 0.87
# model = load_model(data_dir + 'models/aggregate_model_20170511-133235.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170515-062741.h5')
#number_epoch = 2

In [46]:
# Ran into MemoryError when training DAGG_2 with 4 channels at epoch 50.
# To try to get reduce memory usage, limit the number of samples and batch_size

validation_num_samples = min(1280, number_of_samples - split)
x_valid_f2 = x_valid[:validation_num_samples]
y_valid_f2 = y_valid[:validation_num_samples]

# Note: threshold is fixed (not optimized per label)
def compute_f2_measure(l_model):    
    val_generator_f2 = validation_datagen.flow(
        x_valid_f2,
        y_valid_f2,
        batch_size=64,
        shuffle=False)
    raw_pred = l_model.predict_generator(val_generator_f2, validation_num_samples)
    thresholded_pred = (np.array(raw_pred) > classifier_threshold).astype(int)
    l_f2_score = fbeta_score(y_valid_f2, thresholded_pred, beta=2, average=score_averaging_method)
    return l_f2_score
    
class F2_Validation(k.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.f2_measures = []
    def on_epoch_end(self, epoch, logs={}):
        self.f2_measures.append(compute_f2_measure(self.model))

f2_score_val = F2_Validation()

In [47]:
# stop overfitting on training data
early_stop = EarlyStopping(monitor='val_loss',patience=3, min_delta=0, verbose=0, mode='auto')  # TODO patience and min_delta

In [48]:
training_start_time = datetime.now()
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(train_generator,
                    samples_per_epoch=num_samples_per_epoch,
                    nb_epoch=number_epoch,
                    validation_data=validation_generator,
                    nb_val_samples=number_validations,
                    callbacks=[f2_score_val, early_stop])

model.save(model_filepath)  # always save your model and weights after training or during training
time_spent_trianing = datetime.now() - training_start_time

print('model training complete')

Epoch 1/2
44179/44179 [==============================] - ETA: 104s - loss: 0.6926 - acc: 0.5449 - recall: 0.4096 - precision: 0.3869 - ETA: 105s - loss: 0.6834 - acc: 0.5879 - recall: 0.2048 - precision: 0.1935 - ETA: 104s - loss: 0.6978 - acc: 0.5794 - recall: 0.1365 - precision: 0.1290 - ETA: 102s - loss: 0.6933 - acc: 0.5835 - recall: 0.1024 - precision: 0.0967 - ETA: 102s - loss: 0.6917 - acc: 0.5813 - recall: 0.0875 - precision: 0.1574

In [49]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 64, 62, 62)    2368        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 64, 62, 62)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 64, 31, 31)    0           activation_5[0][0]               
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 64, 29, 29)    36928       maxpooling2d_5[0][0]             
___________________________________________________________________________________________

In [53]:

# model = load_model(data_dir + 'models/aggregate_model_20170517-062305.h5')
print(y_valid.shape)
print(y_valid.ndim)

(8096L,)
1


In [ ]:
# TODO 
# imgaug rotations to generate 3 additional sets of x_valid: x_valid_90, x_valid_180, x_valid_270
# run predict_generator on each set.
# numpy.maximum on p_valid_0, p_valid_90, p_valid_180, p_valid_270

In [59]:
# use the validation data to compute some stats which tell us how the model is performing on the validation data set.
val_generator_score_board = validation_datagen.flow(
    x_valid,
    y_valid,
    batch_size=batch_size,
    shuffle=False)
p_valid = model.predict_generator(val_generator_score_board, number_validations)

optimized_thresholds = f2_optimized_thresholds(y_valid, np.array(p_valid))

y_predictions = (np.array(p_valid) > optimized_thresholds).astype(int)

precision_s = precision_score(y_valid, y_predictions, average=score_averaging_method)
print('>>>> Overall precision score over validation set ' , precision_s)

recall_s = recall_score(y_valid, y_predictions, average=score_averaging_method)
print('>>>> Overall recall score over validation set ' , recall_s)

# F2 score, which gives twice the weight to recall
# 'samples' is what the evaluation criteria is for the contest
f2_score = fbeta_score(y_valid, y_predictions, beta=2, average=score_averaging_method)
print('>>>> Overall F2 score over validation set ' , f2_score)


label:0 threshold:0.38 score:0.653713928534
('>>>> Overall precision score over validation set using samples averaging ', 0.3538201296870595)
('>>>> Overall recall score over validation set using samples averaging ', 0.82947785855915401)
('>>>> Overall F2 score over validation set using samples averaging ', 0.65371392853422228)


In [27]:
threshold_df = pd.DataFrame({'label':labels, 
                             'optimized_threshold':optimized_thresholds})
print(threshold_df)

                label  optimized_threshold
0          slash_burn                 0.49
1               clear                 0.00
2            blooming                 0.51
3             primary                 0.00
4              cloudy                 0.46
5   conventional_mine                 0.44
6               water                 0.52
7                haze                 0.49
8         cultivation                 0.50
9       partly_cloudy                 0.00
10     artisinal_mine                 0.48
11         habitation                 0.50
12        bare_ground                 0.48
13          blow_down                 0.47
14        agriculture                 0.36
15               road                 0.49
16  selective_logging                 0.51


In [28]:
precision_l, recall_l, f2_score_l = calculate_stats_for_prediction(y_valid, y_predictions)

prediction_stats_df = pd.DataFrame({
    'label': labels, 
    'true_sum': np.sum(y_valid, axis=0),
    'predict_sum': np.sum(y_predictions, axis=0),
    'f2': f2_score_l,
    'recall': recall_l,
    'precision': precision_l
})

# reordering the columns for easier reading
prediction_stats_df = prediction_stats_df[['label', 'f2', 'recall', 'precision', 'true_sum', 'predict_sum']]
print(prediction_stats_df)

C:\Users\Me\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Me\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


                label    f2  recall  precision  true_sum  predict_sum
0          slash_burn  0.00    0.00       0.00        44            0
1               clear  0.92    1.00       0.70      5684         8096
2            blooming  0.00    0.00       0.00        66            0
3             primary  0.98    1.00       0.92      7451         8096
4              cloudy  0.00    0.00       0.33       452            3
5   conventional_mine  0.00    0.00       0.00        22            0
6               water  0.55    0.83       0.23      1513         5437
7                haze  0.00    0.00       0.00       527            0
8         cultivation  0.00    0.00       0.00       874            0
9       partly_cloudy  0.52    1.00       0.18      1433         8096
10     artisinal_mine  0.00    0.00       0.00        70            0
11         habitation  0.00    0.00       0.00       732            0
12        bare_ground  0.00    0.00       0.00       170            0
13          blow_dow

In [29]:
filtered_data_mask_label = data_mask_label[data_mask]

data_set_name = os.path.basename(get_training_set_file_path(rescaled_dim))

record_model_scores(model_filename, 
                    model_id, 
                    history, 
                    f2_score, 
                    time_spent_trianing, 
                    num_channels,
                    config_file,
                    np.array_str(filtered_data_mask_label),
                    data_set_name)

In [30]:
figures_dir = 'figures/' + model_id
makedirs(figures_dir)

# list all data in history
print('training history stats:')
print(history.history.keys())

# summarize history for f2 score
fig = plt.figure(figsize=(15, 10))
subplot0 = fig.add_subplot(231)
if hasattr(f2_score_val, 'f2_measures'):
    subplot0.plot(f2_score_val.f2_measures)
subplot0.set_title('f2 score')
subplot0.set_ylabel('f2 score')
subplot0.set_xlabel('epoch')
subplot0.legend(['val'], loc='upper left')

# summarize history for recall
subplot3 = fig.add_subplot(232)
subplot3.plot(history.history['recall'])
subplot3.plot(history.history['val_recall'])
subplot3.set_title('recall')
subplot3.set_ylabel('recall')
subplot3.set_xlabel('epoch')
subplot3.legend(['train', 'val'], loc='upper left')

# summarize history for precision
subplot2 = fig.add_subplot(233)
subplot2.plot(history.history['precision'])
subplot2.plot(history.history['val_precision'])
subplot2.set_title('precision')
subplot2.set_ylabel('precision')
subplot2.set_xlabel('epoch')
subplot2.legend(['train', 'val'], loc='upper left')

# summarize history for accuracy
subplot1 = fig.add_subplot(234)
subplot1.plot(history.history['acc'])
subplot1.plot(history.history['val_acc'])
subplot1.set_title('accuracy')
subplot1.set_ylabel('accuracy')
subplot1.set_xlabel('epoch')
subplot1.legend(['train', 'val'], loc='upper left')

# summarize history for loss
subplot4 = fig.add_subplot(235)
subplot4.plot(history.history['loss'])
subplot4.plot(history.history['val_loss'])
subplot4.set_title('model loss')
subplot4.set_ylabel('loss')
subplot4.set_xlabel('epoch')
subplot4.legend(['train', 'val'], loc='upper left')

# precision and recall for each label
subplot5 = fig.add_subplot(236)
colors = cm.rainbow(np.linspace(0, 1, len(prediction_stats_df['label'])))
subplot5.scatter(prediction_stats_df['precision'], prediction_stats_df['recall'], c=colors)
subplot5.set_title('precision & recall')
subplot5.set_xlabel('precision')
subplot5.set_ylabel('recall')
for i, txt in enumerate(prediction_stats_df['label']):
    subplot5.annotate(txt, (prediction_stats_df['precision'][i], prediction_stats_df['recall'][i]))

fig.savefig(figures_dir + '/stats_' + timestr + '.png')
#plt.show()

training history stats:
['acc', 'loss', 'recall', 'precision', 'val_acc', 'val_recall', 'val_precision', 'val_loss']


In [6]:
# load pre-trained model
#model = load_model(data_dir + 'models/aggregate_model_20170521-141533.h5')
#print(model.summary())

# copied manually from stout
#recorded_thresholds = [0.13, 0.13, 0.1, 0.14, 0.05, 0.23, 0.17, 0.15, 0.18, 0.23, 0.09, 0.21, 0.17, 0.10, 0.20, 0.23, 0.1]

#image_generator = get_img_generator()




____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 126, 126)  2368        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 126, 126)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 63, 63)    0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 61, 61)    36928       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [ ]:
# this is the configuration we will use for testing:
testset_datagen = image_generator.getTestGenenerator()

In [7]:
if not is_test_set_in_cache(rescaled_dim):
    # populate the test dataset cache
    df_test = pd.read_csv(sample_submission_filepath)
    load_test_set(df_test, rescaled_dim)

submit_df = submission_dataframe(model,
                                 optimized_thresholds,
                                 data_mask,
                                 testset_datagen, 
                                 rescaled_dim, 
                                 labels, 
                                 sample_submission_filepath,
                                 need_norm_stats)
submit_df.to_csv(data_dir + 'my_submissions/submission_' + timestr + '.csv', index=False)

number of test samples:61191
full batches:61 reminder:191


100%|██████████████████████████████████████████████████████████████████████████| 61191/61191 [00:56<00:00, 1074.54it/s]


In [33]:
#test code
# nums_ones = np.ones((1, 17))
# nums_zeros = np.zeros((1, 17))
# haha = np.array([[1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]])

# y_testset_predictions = haha
# result = pd.DataFrame(y_testset_predictions, columns = labels)

# preds = []
# for i in tqdm(range(result.shape[0]), miniters=1000):
#     a = result.ix[[i]]
#     #print(a)
#     a = a.transpose()
#     print(a)
#     a = a.loc[a[i] == 1]
#     print(a)
#     ' '.join(list(a.index))
#     preds.append(' '.join(list(a.index)))
    
# print(preds)

In [35]:
total_exec_time = datetime.now() - start_time
print ('time spent to complete execution: {}'.format(total_exec_time))

time spent to complete execution: 0:05:27.612000
